# Scenario 6: Sub Stores - Memory Partitioning

This scenario demonstrates powermem's sub stores feature - partitioning different types of memories into separate storage for more efficient querying and management.

## Prerequisites

- Completed Scenario 1
- Installed powermem
- Configured OceanBase database (or other storage backend that supports sub stores)
- Configured LLM and Embedding services

## Understanding Sub Stores

Sub stores allow you to:
- Store different types of memories in independent tables
- Configure independent embedding dimensions and services for each sub store
- Automatically route to the correct storage based on metadata
- Migrate existing data to sub stores
- Improve query performance and resource utilization

## ⚠️ Important: Sub Store Activation

**Before you can use sub stores, you MUST call `migrate_to_sub_store()` at least once for each sub store, even if you have no data to migrate.** This initializes the sub store and marks it as ready for use.

```python
# Even with no data to migrate, you must activate each sub store:
memory.migrate_all_sub_stores(delete_source=False)
```

After activation:
- New memories with matching metadata will be automatically routed to the sub store
- Queries with matching filters will automatically route to the sub store
- Without this activation step, sub stores remain dormant and unused

## Step 1: Configure Sub Stores

First, let's create a Memory instance with sub stores:

In [ ]:
from powermem import Memory
import os

# Configure main storage and sub stores
config = {
    "database": {
        "provider": "oceanbase",
        "config": {
            "collection_name": "main_memories",
            "embedding_model_dims": 1536,
            "host": os.getenv("OCEANBASE_HOST", "127.0.0.1"),
            "port": int(os.getenv("OCEANBASE_PORT", "2881")),
            "user": os.getenv("OCEANBASE_USER", "root@test_tenant"),
            "password": os.getenv("OCEANBASE_PASSWORD", ""),
            "db_name": os.getenv("OCEANBASE_DATABASE", "powermem"),
        }
    },
    "llm": {
        "provider": "qwen",
        "config": {
            "model": "qwen-max",
            "api_key": os.getenv("DASHSCOPE_API_KEY", "your-api-key"),
        }
    },
    "embedder": {
        "provider": "qwen",
        "config": {
            "model": "text-embedding-v4",
            "embedding_dims": 1536,
            "api_key": os.getenv("DASHSCOPE_API_KEY", "your-api-key"),
        }
    },
    # Configure sub stores
    "sub_stores": [
        {
            "collection_name": "working_memories",
            "routing_filter": {"memory_type": "working"},
            "embedding_model_dims": 1536,
        },
        {
            "collection_name": "episodic_memories",
            "routing_filter": {"memory_type": "episodic"},
            "embedding_model_dims": 1536,
        }
    ]
}

# Initialize Memory
memory = Memory(config=config)
print("✓ Memory initialized successfully with 2 sub stores")
print("  - Main store: main_memories (for semantic memories)")
print("  - Sub store 0: working_memories (for working memories)")
print("  - Sub store 1: episodic_memories (for episodic memories)")

## Step 2: Add Different Types of Memories

Let's add different types of memories to the main store:

In [ ]:
user_id = "demo_user"

# Add semantic memories (long-term knowledge, stays in main store)
print("1. Adding semantic memories (knowledge)...")
memory.add(
    messages="Python is a high-level programming language known for its simplicity",
    metadata={"memory_type": "semantic", "topic": "programming"},
    user_id=user_id
)
print("  ✓ Added")

# Add working memories (short-term tasks)
print("\n2. Adding working memories (tasks)...")
memory.add(
    messages="Meeting scheduled at 3 PM today",
    metadata={"memory_type": "working", "importance": "medium"},
    user_id=user_id
)
print("  ✓ Added")

# Add episodic memories (personal experiences)
print("\n3. Adding episodic memories (experiences)...")
memory.add(
    messages="Last summer I visited Paris and saw the Eiffel Tower",
    metadata={"memory_type": "episodic", "time": "2024-07"},
    user_id=user_id
)
print("  ✓ Added")

print("\n✓ Memories added (currently all in main store)")

## Step 3: Migrate Data to Sub Stores (REQUIRED)

Now let's migrate data to the respective sub stores. **This step is mandatory to activate sub stores:**

In [ ]:
print("Starting data migration to sub stores...\n")

# Migrate working memories to sub store 0
# ⚠️ IMPORTANT: This call is REQUIRED even if you have no data to migrate!
# It activates the sub store and marks it as ready for use.
print("1. Migrating working memories to sub store 0...")
working_count = memory.migrate_to_sub_store(
    sub_store_index=0,
    delete_source=True  # Delete from main store after migration
)
print(f"  ✓ Migrated {working_count} working memories")
print(f"  ✓ Sub store 0 is now ACTIVE and ready for routing")

# Migrate episodic memories to sub store 1
print("\n2. Migrating episodic memories to sub store 1...")
episodic_count = memory.migrate_to_sub_store(
    sub_store_index=1,
    delete_source=True
)
print(f"  ✓ Migrated {episodic_count} episodic memories")
print(f"  ✓ Sub store 1 is now ACTIVE and ready for routing")

print("\n✓ Migration completed!")

### 💡 Important Note About Sub Store Activation

**Why is `migrate_to_sub_store()` required?**

1. **Initialization**: Sub stores are created during Memory initialization, but they start in a dormant state
2. **Activation**: Calling `migrate_to_sub_store()` marks the sub store as "ready" and enables routing
3. **No data required**: You can call it with `delete_source=False` even if there's nothing to migrate
4. **One-time operation**: Once activated, the sub store remains active for all future operations

**Without calling `migrate_to_sub_store()`:**
- Sub stores exist but are not used
- All new memories go to the main store
- Queries don't route to sub stores
- The routing filters are ignored

**After calling `migrate_to_sub_store()`:**
- Sub store is marked as active and ready
- New memories automatically route based on metadata
- Queries automatically route based on filters
- The sub store is fully functional

## Step 4: Query After Migration (Automatic Routing)

After migration, queries automatically route to the correct sub store:

In [ ]:
print("Querying after migration (automatic routing)\n")

# Query working memories (should route to sub store 0)
print("1. Querying working memories (routes to sub store 0)...")
results = memory.search(
    query="today's schedule",
    filters={"memory_type": "working"},
    user_id=user_id,
    limit=5
)
results_list = results.get("results", [])
print(f"  Found {len(results_list)} results")
for i, result in enumerate(results_list, 1):
    source = result.get('_source_store', 'unknown')
    print(f"  {i}. [{source}] {result['memory'][:50]}")

# Query episodic memories (should route to sub store 1)
print("\n2. Querying episodic memories (routes to sub store 1)...")
results = memory.search(
    query="past memories",
    filters={"memory_type": "episodic"},
    user_id=user_id,
    limit=5
)
results_list = results.get("results", [])
print(f"  Found {len(results_list)} results")
for i, result in enumerate(results_list, 1):
    source = result.get('_source_store', 'unknown')
    print(f"  {i}. [{source}] {result['memory'][:50]}")

# Query semantic memories (should query main store)
print("\n3. Querying semantic memories (queries main store)...")
results = memory.search(
    query="programming knowledge",
    filters={"memory_type": "semantic"},
    user_id=user_id,
    limit=5
)
results_list = results.get("results", [])
print(f"  Found {len(results_list)} results")
for i, result in enumerate(results_list, 1):
    source = result.get('_source_store', 'main')
    print(f"  {i}. [{source}] {result['memory'][:50]}")

## Step 5: Add New Memories (Automatic Routing)

New memories are automatically routed to the correct sub store (because we activated them in Step 3):

In [ ]:
print("Adding new memories (testing automatic routing)\n")

# Add new working memory (should route to sub store 0)
print("1. Adding new working memory...")
memory.add(
    messages="Remember to call the dentist tomorrow morning",
    metadata={"memory_type": "working", "importance": "high"},
    user_id=user_id
)
print("  ✓ Automatically routed to sub store 0")

# Add new episodic memory (should route to sub store 1)
print("\n2. Adding new episodic memory...")
memory.add(
    messages="I graduated from university in 2020",
    metadata={"memory_type": "episodic", "time": "2020"},
    user_id=user_id
)
print("  ✓ Automatically routed to sub store 1")

# Add new semantic memory (should stay in main store)
print("\n3. Adding new semantic memory...")
memory.add(
    messages="Docker is a platform for developing and deploying containerized applications,I love using Docker",
    metadata={"memory_type": "semantic", "topic": "technology"},
    user_id=user_id
)
print("  ✓ Automatically routed to main store")

print("\n✓ All new memories correctly routed to their respective stores")

## Summary

In this scenario, we learned:
- ✓ How to configure and initialize sub stores
- ✓ Adding different types of memories
- ✓ **REQUIRED**: Calling `migrate_to_sub_store()` to activate sub stores
- ✓ Migrating data to sub stores
- ✓ Automatic query routing
- ✓ Automatic routing for new memories

### Key Takeaway: Sub Store Activation

**Remember**: Sub stores must be explicitly activated by calling `migrate_to_sub_store()` at least once, even if you have no data to migrate. Without this activation step, sub stores remain dormant and unused, and all operations will continue using only the main store.

### Sub stores Benefits

- Optimize storage costs (different embedding dimensions)
- Improve query performance (data partitioning)
- Better data organization (separation by type)
- Flexible management strategies (independent configuration)